In [20]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call
from tqdm import tqdm
from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import subprocess

command = ["sleep", "10s"]
process = subprocess.run(command)
print("a")

a


In [8]:
subprocess.run(["sleep", "10s"])

CompletedProcess(args=['sleep', '10s'], returncode=0)

In [26]:
get_current_folders("dropbox")

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


analysed:   0%|          | 0/249 [00:00<?, ?it/s]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id
6913,/projects/0/einf914/data/20201124_0000_Plate39,94,39,Carrot,A5,001P100N,Y,NaN,20201123,True,...,True,False,True,True,True,True,"24.11.2020, 00:00:",20201124_0000_Plate39,25.0,9420201123
6994,/projects/0/einf914/data/20201210_1156_Plate39,94,39,Carrot,A5,001P100N,Y,NaN,20201123,True,...,False,True,True,True,True,False,"10.12.2020, 11:56:",20201210_1156_Plate39,25.0,9420201123
5992,/projects/0/einf914/data/20201124_0301_Plate39,94,39,Carrot,A5,001P100N,Y,NaN,20201123,True,...,True,False,True,True,True,True,"24.11.2020, 03:01:",20201124_0301_Plate39,25.0,9420201123
6671,/projects/0/einf914/data/20201123_2300_Plate39,94,39,Carrot,A5,001P100N,Y,NaN,20201123,True,...,True,False,True,True,True,True,"23.11.2020, 23:00:",20201123_2300_Plate39,25.0,9420201123
7268,/projects/0/einf914/data/20201124_1001_Plate39,94,39,Carrot,A5,001P100N,Y,NaN,20201123,True,...,True,False,True,True,True,True,"24.11.2020, 10:01:",20201124_1001_Plate39,25.0,9420201123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842,/projects/0/einf914/data/20210930_2123_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,False,True,True,True,False,"30.09.2021, 21:23:",20210930_2123_Plate21,NaN,79220210915
6350,/projects/0/einf914/data/20211005_0904_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,False,True,True,True,False,"05.10.2021, 09:04:",20211005_0904_Plate21,NaN,79220210915
6591,/projects/0/einf914/data/20211007_1548_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,False,True,True,True,False,"07.10.2021, 15:48:",20211007_1548_Plate21,NaN,79220210915
7684,/projects/0/einf914/data/20210928_0308_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,False,True,True,True,False,"28.09.2021, 03:08:",20210928_0308_Plate21,NaN,79220210915


In [24]:
from tqdm.notebook import trange, tqdm
from time import sleep

for i in trange(3, desc="1st loop"):
    for j in tqdm(range(100), desc="2nd loop", leave=False):
        sleep(0.1)

1st loop:   0%|          | 0/3 [00:00<?, ?it/s]

2nd loop:   0%|          | 0/100 [00:00<?, ?it/s]

2nd loop:   0%|          | 0/100 [00:00<?, ?it/s]

2nd loop:   0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
path_bash = os.getenv("HOME") + "/bash/"
path_job = f"{path_bash}transfer.sh"

# call(f"RES=$(sbatch {path_job})", shell=True)

In [25]:
call(f"$(sbatch {path_job})", shell=True)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
/bin/sh: Submitted: command not found


127

In [10]:
f"(sbatch {path_job})"

'(sbatch /home/cbisot/bash/transfer.sh)'

In [3]:
import subprocess

output = subprocess.check_output(["sbatch", f"{path_job}"])

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [9]:
string = output.decode("utf-8")
string.split(" ")[-1][:-2]

'24841'

In [24]:
stra = os.getenv("RES")
stra